In [1]:
import cv2
import numpy as np

print(cv2.__version__)
print(np.__version__)

def resize_image(image, target_size=(800, 1000)):
  '''Resize the image while maintaining aspect ratio'''
  h, w = image.shape[:2]
  scale = min(target_size[1] / w, target_size[0] / h)
  new_w = int(w * scale)
  new_h = int(h * scale)
  resized_image = cv2.resize(image, (new_w, new_h))
  return resized_image

def logarithmic_transformation(image, epsilon=1e-5):
  '''Apply logarithmic transformation to the image with zero value handling'''
  c = 255 / np.log(1 + np.max(image))
  # Epsilon zero-handling technique
  log_image = c * (np.log(1 + image + epsilon))
  log_image = np.array(log_image, dtype=np.uint8)

  return log_image

def contrast_stretching(image):
  min_val = np.min(image)
  max_val = np.max(image)
  stretched = (image - min_val) * (255 / (max_val - min_val))
  return stretched.astype(np.uint8)

def gaussian_blur(image, mode='Default'):
  if mode == 'Default':
    kernel_size = (3,3)
  elif mode == 'Medium':
    kernel_size = (5,5)
  elif mode == 'Hard':
    kernel_size = (7,7)
  else:
    raise ValueError("Mode must be 'Default', 'Medium', or 'Hard'")

  return cv2.GaussianBlur(image, kernel_size, 0)

def otsu_thresholding(image):
  _, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
  return binary_image

def canny_edge_detection(image, low_threshold=50, high_threshold=150):
  return cv2.Canny(image, low_threshold, high_threshold)

def find_extreme_corners(contours):
  '''Find the extreme corners of the image'''
  all_points = np.vstack(contours)
  top_left = all_points[np.argmin(all_points[:, :, 0] + all_points[:, :, 1])]
  bottom_right = all_points[np.argmax(all_points[:, :, 0] + all_points[:, :, 1])]
  top_right = all_points[np.argmax(all_points[:, :, 0] - all_points[:, :, 1])]
  bottom_left = all_points[np.argmin(all_points[:, :, 0] - all_points[:, :, 1])]
  return top_left[0], top_right[0], bottom_left[0], bottom_right[0]

def apply_perspective_transformation(image, corners):
  '''Apply perspective transformation to the image'''
  tl, tr, bl, br = corners
  width = int(max(np.linalg.norm(br - bl), np.linalg.norm(tr - tl)))
  height = int(max(np.linalg.norm(tr - br), np.linalg.norm(tl - bl)))

  dst_pts = np.array([
      [0, 0],
      [width - 1, 0],
      [0, height - 1],
      [width - 1, height - 1]
  ], dtype="float32")

  src_pts = np.array([tl, tr, bl, br], dtype="float32")

  M = cv2.getPerspectiveTransform(src_pts, dst_pts)
  warped = cv2.warpPerspective(image, M, (width, height))
  return warped

def automatic_warp_transformation(image, target_size=(800, 1000)):
  '''Automatic Cropping using Adaptive Warp Transformation'''
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  resized_image = resize_image(gray_image, target_size)
  brightened_image = logarithmic_transformation(resized_image)
  contrast_image = contrast_stretching(brightened_image)
  blurred_image = gaussian_blur(contrast_image, mode='Default')
  binary_image = otsu_thresholding(blurred_image)
  edges = canny_edge_detection(binary_image)
  contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Getting Contours (Drawing Contours in image, useful for debugging)
  contour_image = cv2.cvtColor(binary_image, cv2.COLOR_GRAY2BGR)
  cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)

  corners = find_extreme_corners(contours)
  for corner in corners:
      cv2.circle(contour_image, tuple(corner), 5, (0, 0, 255), -1)

  warped_image = apply_perspective_transformation(resized_image, corners)
  print(f'Initial image {image.shape} processed to {warped_image.shape}')

  return warped_image

def image_uniformization(master_image, student_image):
  '''Precision Image Resizing'''
  master_shape = master_image.shape
  student_shape = student_image.shape

  master_height = master_shape[0]
  master_width = master_shape[1]

  student_height = student_shape[0]
  student_width = student_shape[1]

  min_height = min(master_height, student_height)
  min_width = min(master_width, student_width)

  resized_master = cv2.resize(master_image, (min_width, min_height))
  resized_student = cv2.resize(student_image, (min_width, min_height))

  print(f'master_key {master_image.shape} and student_answer {student_image.shape} uniformed to {resized_master.shape}')

  return resized_master, resized_student

def morph_open(image):
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
  eroded_img = cv2.erode(image, kernel, iterations = 1)
  dilated_img = cv2.dilate(eroded_img, kernel, iterations = 1)

  return dilated_img

def core_circles_preprocessing(image):
  '''Core Circles Preprocessing Module'''
  blurred_img = gaussian_blur(image, mode='Hard')
  contrast_img = contrast_stretching(blurred_img)
  log_img = logarithmic_transformation(contrast_img)
  binary_img = otsu_thresholding(log_img)
  opened_img = morph_open(binary_img)

  return opened_img

def draw_full_contours(contours, cont_image, radius = 7):
  '''Draw Full Circles'''
  for contour in contours:
    M = cv2.moments(contour)
    if M["m00"] != 0:
      cX = int(M["m10"] / M["m00"])
      cY = int(M["m01"] / M["m00"])
      # Draw a filled circle at the center of the contour
      cv2.circle(cont_image, (cX, cY), radius, (0, 255, 0), -1)

  return cont_image

def extract_and_draw_contours(image):
  contours, hierarchy = cv2.findContours(image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

  unique_values = []
  for columns in image:
    for pixel in columns:
      if pixel not in unique_values:
        unique_values.append(pixel)

  contour_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

  # # draw filled circles
  # cv2.drawContours(contour_image, contours, -1, (0, 255, 0), thickness = -1)
  # # draw hollow circles
  # cv2.drawContours(contour_image, contours, -1, (0, 255, 0), thickness = 2)

  # draw full contours
  contour_image = draw_full_contours(contours, contour_image)

  return contours, contour_image

def final_scoring(new_student, processed_student, master_contours):
  '''Final Score Calculation'''
  test_answer = processed_student.copy()
  # drawing the Answer Key to the Student's test answer, extracting the mistakes information
  check_answers = draw_full_contours(master_contours, test_answer)

  # open the image to remove noise
  final_sheet = morph_open(check_answers)

  # fetching mistakes contours
  final_contours, _ = extract_and_draw_contours(final_sheet)

  # calculating mistakes and final score
  mistakes = len(final_contours)
  total_questions = len(master_contours)
  final_score = ((total_questions - mistakes) / total_questions) * 100
  print(f'final score: {final_score}')

  student_correction = cv2.cvtColor(new_student, cv2.COLOR_GRAY2BGR)
  student_correction = draw_full_contours(master_contours, student_correction)

  return final_score, student_correction

def show_image(image): 
  cv2.imshow('image', image)
  cv2.waitKey(0)
  cv2.destroyAllWindows()

4.9.0
1.26.4


In [ ]:
answer1 = cv2.imread('inputs/circle_1/stu_good_light.jpg')
master_sheet = cv2.imread('inputs/circle_1/master_circle_best.jpg')

# answer1 = cv2.imread('inputs/circle_2/student.jpg')
# master_sheet = cv2.imread('inputs/circle_2/master.jpg')

# show_image(answer1)

In [3]:
student1 = automatic_warp_transformation(answer1)
master_key = automatic_warp_transformation(master_sheet)

Initial image (3839, 1036, 3) processed to (776, 188)
Initial image (3832, 1021, 3) processed to (781, 189)


In [4]:
new_master, new_student = image_uniformization(master_key, student1)

master_key (781, 189) and student_answer (776, 188) uniformed to (776, 188)


In [5]:
processed_master = core_circles_preprocessing(new_master)
processed_student = core_circles_preprocessing(new_student)

# cv2_imshow(processed_master)

In [6]:
student_contours, student_contour_image = extract_and_draw_contours(processed_student)
master_contours, master_contour_image = extract_and_draw_contours(processed_master)

print(len(student_contours))
print(len(master_contours))

# cv2_imshow(student_contour_image)

# contours, hierarchy = cv2.findContours(processed_student, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

# unique_values = []
# for columns in processed_student:
#   for pixel in columns:
#     if pixel not in unique_values:
#       unique_values.append(pixel)

# print(len(contours))
# print(type(contours))
# print(unique_values)

# # Draw the contours on a copy of the original image
# contour_image = cv2.cvtColor(processed_student, cv2.COLOR_GRAY2BGR)  # Convert to BGR for color drawing
# cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 3)  # Green color for contours

# # Display the image with contours
# cv2_imshow(contour_image)

100
100


In [7]:
stu_final_score, stu_answer_key = final_scoring(new_student, processed_student, master_contours)
cv2.imshow("student's answer vs master key: ", stu_answer_key)
cv2.waitKey(0)
cv2.destroyAllWindows()

final score: 86.0
